# SECURITIES BALANCE SHEET

Library

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import datetime as dt

Symbol

In [2]:
# Save as a table
sql_symbol = pd.read_csv("symbol_id.csv")
sql_symbol.drop(['Unnamed: 0'], axis=1, inplace=True)

# Save as a dict
dict_sql_symbol = dict(zip(sql_symbol['SymbolID'], sql_symbol['Symbol']))

Pathlink

In [3]:
# Assign pathlink
algo_pathlink = r"/Users/rainmeteror/Desktop/BSC_Algo_Database_UAT/OneDrive_1_5-29-2023/securities/algo_sbs.xlsx"
sql_pathlink = r"/Users/rainmeteror/Desktop/BSC_Algo_Database_UAT/OneDrive_1_5-29-2023/securities/sql_sbs.xlsx"
same_field_data = r"/Users/rainmeteror/Desktop/BSC_Algo_Database_UAT/OneDrive_1_5-29-2023/securities/sbs_datafield.xlsx"

Algo Database

In [4]:
# Read data from algo database
df_algo = pd.read_excel(algo_pathlink)
df_algo.drop(['Unnamed: 0', 'SCN', 'REPORTED_DATE', 'AUDITED', 'MONTH_IN_PERIOD', 'CREATED_DATE', 'UPDATED_DATE'], axis=1, inplace=True)
df_algo = df_algo.loc[df_algo['FREQ_CODE'] != 'Y']
df_algo['YEAR'] = df_algo['REPORT_DATE'].dt.year
df_algo['QUARTER'] = df_algo['REPORT_DATE'].dt.quarter
df_algo.drop(['REPORT_DATE', 'FREQ_CODE'], axis=1, inplace=True)

# Sort Values
df_algo.sort_values(by=['YEAR', 'QUARTER', 'SECURITY_CODE'], ascending=[True, True, True], inplace=True)

# Print how many rows are used
print("ALGO DATABASE SECURITIES: There are: " + str(len(df_algo)) + " rows")
print("ALGO DATABASE SECURITIES: There are: " + str(len(df_algo.columns)) + " columns")

ALGO DATABASE SECURITIES: There are: 1017 rows
ALGO DATABASE SECURITIES: There are: 177 columns


In [5]:
df_algo.head(1)

,SECURITY_CODE,SBS_100,SBS_110,SBS_111,SBS_111_1,SBS_111_3,SBS_111_2,SBS_111_4,SBS_112,SBS_113,...,SBS_420_1,SBS_420_2,SBS_420_3,SBS_418_1,SBS_440,SBS_440_1,SBS_440_2,SBS_119,YEAR,QUARTER
0,AAS,366356284852,365863712642,31333663618,31333663618,0.0,0,299558574910,74935886154,198052054800,...,0,0.0,0.0,0,410272838374,0.0,NaN,NaN,2020,1


Stockbiz Database

In [6]:
# Read data from sql database
df_sql = pd.read_excel(sql_pathlink)
df_sql.drop(['Unnamed: 0', 'ReportID', 'LastUpdated', 'ApprovedBy', 'CreatedBy', 'LastUpdatedBy',
             'ReportStatus', 'AssignedTo', 'CreatedAt'], axis=1, inplace=True)
df_sql = df_sql.loc[(df_sql['Quarter'] != 0) & (df_sql['Year'] > 2019)]
df_sql.sort_values(by=['Year', 'Quarter'], ascending=[True, True], inplace=True)

# Change the name of tickers
df_sql['SymbolID'] = df_sql['SymbolID'].map(dict_sql_symbol)

# Change the name of column SymbolID
df_sql.rename(
    columns={"SymbolID": "SECURITY_CODE",
             "Year": "YEAR",
             "Quarter": "QUARTER"},
    inplace=True
)

# Print how many rows are used
print("STOCKBIZ DATABASE SECURITIES: There are: " + str(len(df_sql)) + " rows")
print("STOCKBIZ DATABASE SECURITIES: There are: " + str(len(df_sql.columns)) + " rows")

STOCKBIZ DATABASE SECURITIES: There are: 450 rows
STOCKBIZ DATABASE SECURITIES: There are: 129 rows


In [7]:
df_sql.head(1)

,SECURITY_CODE,QUARTER,YEAR,F1,F2,F3,F4,F5,F6,F7,...,F120,F121,F122,F123,F124,F125,F126,F97,F98,F103
3,SSI,1,2020,21677442390472,21635844678933,282437672957,2.512291e+11,3.120858e+10,6.478018e+12,1.051199e+13,...,3.496463e+12,-5.417922e+11,6.421930e+10,0.0,2.749951e+13,NaN,NaN,5.313716e+10,0.0,0.0


In [8]:
# Get Symbol in insurance sectors
securities_symbol = df_sql['SECURITY_CODE'].unique()

In [9]:
print("There are: "+ str(len(securities_symbol))+ " stocks")

There are: 36 stocks


Process 2 tables

In [10]:
# Change the name of columns
same_column_name = pd.read_excel(same_field_data)
same_column_name.dropna(axis=0, inplace=True)

# Create a dict with key from bbs_sql and value from bbs_algo
dict_column_name = dict(zip(same_column_name['sbs_sql'], same_column_name['sbs_algo']))

In [11]:
same_column_name.head(1)

,sbs_algo,sbs_sql
0,SBS_100,F1


In [12]:
df_sql.dtypes

SECURITY_CODE     object
QUARTER            int64
YEAR               int64
F1                 int64
F2                 int64
                  ...   
F125             float64
F126             float64
F97              float64
F98              float64
F103             float64
Length: 129, dtype: object

In [13]:
# Change the name of df_sql based on name of the similar data field in df_algo
df_sql.rename(
    columns=dict_column_name,
    inplace=True
)

# Fill NA values in df_sql
df_sql = df_sql.fillna(0)

# Change the type of data which is same between two tables
df_sql[same_column_name['sbs_algo']] = df_sql[same_column_name['sbs_algo']].astype('Int64')

In [14]:
# Change the type of data in df_algo
# Just get the same data between two tables
df_algo[same_column_name['sbs_algo']] = df_algo[same_column_name['sbs_algo']].astype('Int64')

# Change the type of YEAR and QUARTER
df_algo =  df_algo.astype(
    {"YEAR": "Int64",
     "QUARTER": "Int64"},
)

# Get the data with same value in banking sector
df_algo = df_algo.loc[df_algo['SECURITY_CODE'].isin(securities_symbol)]

In [15]:
# Get the final same columns
final_column = ["SECURITY_CODE", "YEAR", "QUARTER"]

for i in same_column_name['sbs_algo']:
    final_column.append(i)

In [16]:
df_sql = df_sql[final_column]
df_algo = df_algo[final_column]

In [17]:
def compare_table(df1, df2, columns) -> pd.DataFrame():
    
    return pd.merge(df1, df2, on=columns, how='outer', indicator=True).query("_merge != 'both'")

In [18]:
final_result = {"field": [], "value_not_same": []}
for i in range(3, len(final_column)):
    check_columns = ["SECURITY_CODE", 'YEAR', 'QUARTER', final_column[i]]
    result = compare_table(
        df1=df_sql[check_columns], 
        df2=df_algo[check_columns], 
        columns=check_columns)
    
    value_not_same = len(result)/(len(df_algo) + len(df_sql))*100
    
    final_result["field"].append(final_column[i])
    final_result["value_not_same"].append(value_not_same)

In [19]:
pd.DataFrame(final_result).to_excel("check_sbs.xlsx")